In [ ]:
import os
import numpy as np
from transformers.models.roberta.tokenization_roberta import RobertaTokenizer
from transformers.optimization import AdamW
from transformers.models.roberta.modeling_roberta import RobertaModel

import torch
import torch.nn as nn
import random
from torch.nn import functional as F

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

### Load Data

In [ ]:
Cbase_pretrain_path = '../Dataset/HWU/Pretrain.txt'
Cbase_x = []
Cbase_y = []
for line in open(Cbase_pretrain_path):
    if len(line.split('\t')) != 2:
        continue
    else:
        Cbase_y.append(line.split('\t')[1][:-1].replace('_',' '))
        Cbase_x.append(line.split('\t')[0])

label_list = list(set(Cbase_y))
label_yd_yi_dic = {}
for i in range(len(label_list)):
    label_yd_yi_dic[label_list[i]] = str(i)

In [ ]:
max_length = 32
n_tokens = 5
hidden_size = 1024
batch_size = 10
cluster_nums = len(list(set(Cbase_y)))
cluster_nums_str = str(cluster_nums)
N = 2500
base_lr = 5e-6
lr_scale = 100

In [ ]:
Cn_path = '../Dataset/HWU/Aux1PMulti.txt'
Cn = []
for line in open(Cn_path):
    Cn.append(line)
random.shuffle(Cn)

Cn_x = []
Cn_y = []
for line in Cn:
    Cn_y.append(line.split('\t')[1][:-1].replace('_',' '))
    Cn_x.append(line.split('\t')[0])

pre_yd_list = label_yd_yi_dic.keys()
pre_yd_yi_dic = label_yd_yi_dic

Cn_y_id = []
for i in range(len(Cn_y)):
    if Cn_y[i] in pre_yd_list:
        Cn_y_id.append(pre_yd_yi_dic[Cn_y[i]])
    else:
        Cn_y_id.append(cluster_nums_str)

In [ ]:
###### 
from torch.utils.data import Dataset
import torch.utils.data as util_data

class AugmentPairSamples(Dataset):
    def __init__(self, train_x, train_y):
        assert len(train_y) == len(train_x)
        self.train_x = train_x
        self.train_y = train_y
        
    def __len__(self):
        return len(self.train_y)

    def __getitem__(self, idx):
        return {'text': self.train_x[idx], 'label': self.train_y[idx]}

train_dataset = AugmentPairSamples(Cn_x, Cn_y_id)
train_loader = util_data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

### 加载 Pretrained Model

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="0,1,2,3"
device = "cuda:3" if torch.cuda.is_available() else "cpu"

In [ ]:
class PREModel(nn.Module):
    def __init__(self, pre_tokenizer, pre_model, device=device, training_flag=True):
        
        super(PREModel, self).__init__()
        self.device = device
        self.classifier_loss1 = nn.BCELoss()
        self.classifier_loss2 = nn.CrossEntropyLoss()
        self.training_flag = training_flag
        self.optimizer = None
        
        ###### SentenceBert Model ###### 
        self.tokenizer = pre_tokenizer
        self.sentbert = pre_model.roberta_single
        
        ####### Classifer Model ######  
        self.classifer = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(512, 1)).to(self.device)
        
    ###### SentenceBert Model ###### 
    def get_embeddings(self, features, pooling="mean"):
        bert_output =  self.sentbert.forward(**features)
        attention_mask = features['attention_mask'].unsqueeze(-1).to(device)
        all_output = bert_output[0]
        mean_output =  torch.sum(all_output*attention_mask, dim=1) / torch.sum(attention_mask, dim=1)
        return mean_output

    def set_optmizer(self, opt):
        self.optimizer = opt
        
    def forward(self, inputs, labels, aggregate=True):
        for s_idx in range(len(labels)): 
            h0 = self.get_embeddings(inputs[s_idx], pooling="mean").to(self.device) 
            y_vector = torch.tensor([0.]*h0.shape[0]).to(self.device) 
            y_idx = int(labels[s_idx])
            y_onehot = y_vector.clone()
            y_onehot[y_idx] = 1.
            
            if s_idx == 0:
                classifier_out = self.classifer(h0).squeeze().unsqueeze(0) 
                y_onehot_all = y_onehot.unsqueeze(0)
            else:
                classifier_out = torch.cat([classifier_out,self.classifer(h0).squeeze().unsqueeze(0)],dim=0) 
                y_onehot_all = torch.cat([y_onehot_all,y_onehot.unsqueeze(0)],dim=0)

        classifier_output = classifier_out
        classifier_sigmode_vector = F.sigmoid(classifier_output)
        classifier_softmax_vector = F.softmax(classifier_output,dim=1)
        cluster_result = [str(torch.argmax(classifier_softmax_vector[i]).item()) for i in range(classifier_softmax_vector.shape[0])]
            
        class_loss_all = torch.tensor(0.0).to(device)
        class_loss_s = 0.
        class_loss_s1 = self.classifier_loss1(classifier_sigmode_vector,y_onehot_all)
        class_loss_s2 = self.classifier_loss2(classifier_softmax_vector,y_onehot_all)
        class_loss_all += class_loss_s1 + class_loss_s2
        class_loss = torch.div(class_loss_all,h0.shape[0])
        
        if self.training_flag:
            loss = class_loss
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
        return class_loss.detach(), class_loss_s1.detach(), class_loss_s2.detach(), labels, cluster_result
    

In [ ]:
ckpt = '../Result/PreModel.pt'
pre_model = torch.load(ckpt).to(device)

### Build Model

In [ ]:
class TRAINModel(nn.Module):
    def __init__(self, pre_tokenizer, pre_encoder, pre_classifer, device=device, training_flag=True):
        
        super(TRAINModel, self).__init__()
        self.device = device
        self.classifier_loss1 = nn.BCELoss()
        self.classifier_loss2 = nn.CrossEntropyLoss()
        self.training_flag = training_flag
        self.optimizer = None
        
        ###### SentenceBert Model ###### 
        self.tokenizer = pre_tokenizer
        self.sentbert = pre_encoder
        
        ####### Classifer Model ######  
        self.classifer = pre_classifer
        
    ###### SentenceBert Model ###### 
    def get_embeddings(self, features, pooling="mean"):
        bert_output =  self.sentbert.forward(**features)
        attention_mask = features['attention_mask'].unsqueeze(-1).to(device)
        all_output = bert_output[0]
        mean_output =  torch.sum(all_output*attention_mask, dim=1) / torch.sum(attention_mask, dim=1)
        return mean_output
    
    def get_none_embeddings(self, input_t, input_none, pooling="mean"):
        x_embed = self.sentbert.embeddings.word_embeddings(input_t['input_ids']) 
        x_embed_cls = self.sentbert.embeddings.word_embeddings(input_t['input_ids'])[:,0,:].unsqueeze(1) 
        none_embeddings_inital = self.sentbert.forward(**input_none).last_hidden_state 
        soft_text_prompt = nn.Embedding(n_tokens, hidden_size)
        soft_text_prompt.weight = nn.parameter.Parameter(none_embeddings_inital) 
        return x_embed_cls, x_embed, soft_text_prompt.weight

    def set_optmizer(self, opt):
        self.optimizer = opt
        
    def forward(self, inputs, inputs_text, inputs_none, labels, aggregate=True):
        for s_idx in range(len(labels)): #10
            h0 = self.get_embeddings(inputs[s_idx], pooling="mean").to(self.device) 
            
            y_vector = torch.tensor([0.]*(h0.shape[0]+1)).to(self.device)
            y_idx = int(labels[s_idx])
            y_onehot = y_vector.clone()
            y_onehot[y_idx] = 1.
            
            if s_idx == 0:
                classifier_out = self.classifer(h0).squeeze().unsqueeze(0) 
                y_onehot_all = y_onehot.unsqueeze(0)
            else:
                classifier_out = torch.cat([classifier_out,self.classifer(h0).squeeze().unsqueeze(0)],dim=0)
                y_onehot_all = torch.cat([y_onehot_all,y_onehot.unsqueeze(0)],dim=0)
                
        x_embed_cls, x_embed, soft_text_prompt_weight = self.get_none_embeddings(inputs_text.to(self.device),inputs_none.to(self.device))
        x_attmask = inputs_text['attention_mask'] 
        x_ones = torch.ones([batch_size,n_tokens]).to(device) 
        none_attmask = torch.cat([x_ones,x_attmask],dim=1).to(device)

        none_embedding = soft_text_prompt_weight.repeat(batch_size,1,1)
        none_encoderinput = torch.cat([none_embedding,x_embed],dim=1)

        text_input = {
            'inputs_embeds': none_encoderinput,
            'attention_mask': none_attmask.to(device)
        }
        
        h0_none = self.sentbert.forward(**text_input).pooler_output
        classifier_out_none = self.classifer(h0_none)
        classifier_output = torch.cat([classifier_out,classifier_out_none],dim=1)
        classifier_sigmode_vector = F.sigmoid(classifier_output)
        classifier_softmax_vector = F.softmax(classifier_output,dim=1)
        cluster_result = [str(torch.argmax(classifier_softmax_vector[i]).item()) for i in range(classifier_softmax_vector.shape[0])]
        class_loss_all = torch.tensor(0.0).to(device)
        
        class_loss_s = 0.
        class_loss_s1 = self.classifier_loss1(classifier_sigmode_vector,y_onehot_all)
        class_loss_s2 = self.classifier_loss2(classifier_softmax_vector,y_onehot_all)
        class_loss_all += class_loss_s1 + class_loss_s2
        class_loss = torch.div(class_loss_all,h0.shape[0])
        
        ACC_train = accuracy_score(labels,cluster_result)
        F1_train = f1_score(labels, cluster_result, average='macro')
          
        if self.training_flag:
            loss = class_loss
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
        return class_loss.detach(), labels, cluster_result, ACC_train, F1_train

In [ ]:
model = TRAINModel(pre_tokenizer=pre_model.tokenizer, pre_encoder=pre_model.sentbert, pre_classifer=pre_model.classifer, device=device, training_flag=True).to(device)
base_lr = 5e-6
lr_scale = 100
optimizer = torch.optim.Adam([
    {'params':model.sentbert.parameters()}, 
    {'params':model.classifer.parameters(), 'lr':base_lr * lr_scale}], base_lr)

model.set_optmizer(optimizer)

### Training

In [ ]:
def prepare_task_input(batch, is_contrastive=False):
    if is_contrastive:
        text, label = batch['text'], batch['label']
        all_label_descrip = list(label_yd_yi_dic.keys())
        feat = {'input_ids': torch.tensor(0),'attention_mask':torch.tensor(0)}
        features1 = model.tokenizer.batch_encode_plus(text, max_length=max_length, return_tensors='pt', padding='longest', 
                                                         truncation=True)
        features2 = model.tokenizer.batch_encode_plus(all_label_descrip, max_length=max_length, return_tensors='pt', padding='longest', 
                                                         truncation=True)
        features_none = model.tokenizer.batch_encode_plus(['none of them'], max_length=max_length, return_tensors='pt', padding='longest', 
                                                         truncation=True)
        
        feat1_inputids = features1["input_ids"]
        feat1_attmask = features1["attention_mask"]
        feat2_inputids = features2["input_ids"]
        feat2_attmask = features2["attention_mask"]
        max_len = feat1_inputids.shape[1] + feat2_inputids.shape[1] 
        
        # count
        non_zero1 = torch.count_nonzero(feat1_attmask, dim=1).reshape(-1, 1)
        non_zero2 = torch.count_nonzero(feat2_attmask, dim=1).reshape(-1, 1)
        
        feats = []
        for i in range(non_zero1.shape[0]):
            index1 = non_zero1[i][0]
            feat_s = {'input_ids': torch.tensor(0),'attention_mask':torch.tensor(0)}
            for j in range(non_zero2.shape[0]):
                index2 = non_zero2[j][0]
                feat_inputid = torch.cat([feat2_inputids[j][:index2],feat1_inputids[i][1:index1]])
                feat_attmask = torch.tensor([1]*(index1+index2)+[0]*(max_len-index1-index2)).unsqueeze(0)
                feat_inputid = torch.cat([feat_inputid,torch.tensor([1]*(max_len-index1-index2+1))]).unsqueeze(0)
                if j == 0:
                    feat_inputid_s = feat_inputid
                    feat_attmask_s = feat_attmask
                elif j != 0:
                    feat_inputid_s = torch.cat([feat_inputid_s, feat_inputid],dim=0)
                    feat_attmask_s = torch.cat([feat_attmask_s, feat_attmask],dim=0)
            feat_s['input_ids'] = feat_inputid_s.to(device)
            feat_s['attention_mask'] = feat_attmask_s.to(device)
            feats.append(feat_s)  
        
        return feats, features1, features_none, label
    
ClassLoss_all = []
ACC_result = []
F1_result = []
def training(train_loader):
    pre_acc = 0.
    epoch = 0
    for i in np.arange(N):
        model.train()
        model.training_flag = True        
        print('************'+str(i)+'************')        
        try:
            batch = next(train_loader_iter)
        except:
            train_loader_iter = iter(train_loader)
            batch = next(train_loader_iter)      
        
        feats, features1, features_none, label = prepare_task_input(batch, is_contrastive=True)    
        losses = model.forward(feats, features1, features_none, label, aggregate=True)
        
        ACC_result.append(losses[3])
        F1_result.append(losses[4])
        
        ClassLoss_all.append(losses[0])
        
        ###### Save Model ###### 
        best_acc = 0.
        best_ckpt = ''
        if (i+1) % 100 == 0:  # 76
            ckpt = '../Result/Model.pt'
            torch.save(model, ckpt)
    return None       

In [ ]:
training(train_loader)